### Packages

In [1]:
# NumCosmo:
try:
  import gi
  gi.require_version('NumCosmo', '1.0')
  gi.require_version('NumCosmoMath', '1.0')
except:
  pass

from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

import matplotlib.pyplot as plt
import scipy
import numpy as np
from sympy import *
import scipy.stats as scs

# Necessary functions:

from numpy import sqrt, log10, sinh, exp
from numpy import log as ln

#Calculus:
from scipy.integrate import quad #Integrate
from sympy import diff, Symbol, symbols #Derivative
from scipy.optimize import minimize

c = 3*(10**5)
#c =1

### NumCosmo Settings

In [2]:
Ncm.cfg_init ()
cosmo = Nc.HICosmoDEXcdm() #Cosmology object

### Data

In [3]:
ser = Ncm.Serialize.new(0)
data = ser.from_file("/home/cinthia/NumCosmo/data/nc_data_snia_diag_legacy.obj")
lenz = data.y.len()

muobs = []
sigma = []
zobs = []
for i in range(lenz):
    mi = data.y.get(i)
    si = data.sigma.get(i)
    zi = data.x.get(i)
    
    muobs.append(mi)
    sigma.append(si)
    zobs.append(zi)

muobs = np.array(muobs)
zobs = np.array(zobs)
sigmaobs = np.array(sigma)

### A) Likelihood (-2ln L) ($\Omega_{k0} = \Omega_{r0} = 0$ )

In [4]:
def Dl(z, theta):
#Theta
    H0, Omega_m0, Omega_l0  = theta[0], theta[1], theta[2]

#Hubble function
#    E2z = Omega_l0 + Omega_k0*((1 + z) ** 2) + Omega_m0*((1 + z) ** 3) + Omega_r0*((1 + z) ** 4) 
#    Ez = E2z ** 0.5
    

#Comoving distance
    def integ(b):

        int_arg = lambda z: 1/((Omega_l0 + (Omega_m0 * ((1 + z) ** 3))) ** 0.5)
        result = quad(int_arg, 0, b)
        return result[0]

    dc_list = []
    for i in range(lenz):
        d = integ (z[i])     #(* c/H0)   
        dc_list.append(d)

    dc = np.array(dc_list)      
    Dc =  dc     #( * H0/c)


#Temporal comoving distance
#     if Omega_k0 == 0:
#         Dt = Dc
#     else:
#         Dt = sinh((sqrt(Omega_k0)) * Dc) / sqrt(Omega_k0)
    Dt = Dc

    
#Luminosity distance
    D_l = (1 + z) * Dt

    return D_l

def mu(z, theta): #This function calculates the modular distance
#Modular distance
    H0, Omega_m0, Omega_l0  = theta[0], theta[1], theta[2]
    DL = Dl(z, theta)
    mu = ( 5 * log10(DL) ) + 25 + ( 5 * log10(c / H0))
    
    return mu 

def f_2lnL(theta): #This function calculates -2lnL (L is the Likelihood).
    arg = []
    for i in range(lenz):
        argi = ( ( mu(zobs,theta)[i] - muobs[i] ) ** 2 ) / ( sigmaobs[i] ** 2 )
        arg.append(argi)
        
    return sum(arg)  


### Best fit

In [5]:
def fit():
    x0 = [70.0, 0.20, 0.80]
    x0_np = np.array(x0)
    bnds = ((0, None), (0, 1.0), (0, 1.0))
    
    f = minimize(f_2lnL, x0_np, bounds = bnds, method='Nelder-Mead', tol=1e-6)
    
    return f

fit = fit()

print(fit)

 final_simplex: (array([[73.36421072,  0.23935098,  0.67269584],
       [73.36421087,  0.23935098,  0.67269584],
       [73.36421171,  0.23935097,  0.67269582],
       [73.36421065,  0.23935098,  0.67269583]]), array([110.99870476, 110.99870476, 110.99870476, 110.99870476]))
           fun: 110.99870476451699
       message: 'Optimization terminated successfully.'
          nfev: 167
           nit: 84
        status: 0
       success: True
             x: array([73.36421072,  0.23935098,  0.67269584])


### Fisher matrix

In [6]:
#Parameters and data
theta = fit.x
H0 = theta[0]
z = zobs

# factor: del(dc)/delOmega_lambda/m
def dOm(z, Omega):
    dc_list = []
    for i in range(lenz):
        
        H0, Omega_m0, Omega_l0  = theta[0], theta[1], theta[2]
        
        if Omega == 1:        #Omega_Lambda case
            int_arg = lambda z: -(0.5) * ((Omega_l0 + (Omega_m0 * ((1 + z) ** 3)))) ** (3/2)
            d = quad(int_arg, 0, z[i])
        
        else:                #Omega_m case
            
            int_arg = lambda z: -((1 + z) ** 3) * (0.5) * ((Omega_l0 + (Omega_m0 * ((1 + z) ** 3)))) ** (3/2)
            d = quad(int_arg, 0, z[i])
        
               
        dc_list.append(d[0])

    return np.array(dc_list)


#delmu/delH0
delmuH0 =  []

#delmu/delOmegalambda
delmuOL = []

#delmu/delOmegam
delmuOM = []

for k in range(lenz):
    delmuH0i =(- 5) / (H0 * ln(10))
    delmuOLi = (5 / (Dl(z, theta)[k] * ln(10)) * dOm(zobs, 1)[k])
    delmuOMi = (5 / (Dl(z, theta)[k] * ln(10)) * dOm(zobs, 2)[k])
    delmuOL.append(delmuOLi)
    delmuOM.append(delmuOMi)
    delmuH0.append(delmuH0i)


der = [delmuH0, delmuOL, delmuOM] #list of derivatives from mu

#Fisher Matrix
list_mn = []
for n in der:
    list_m = []
    for m in der:
        list_d = []
        for i in range(lenz):
            di = (1 / (sigma[i]) ** 2) * ( n[i] * m[i])
            list_d.append(di)
        list_m.append(sum(list_d))
    list_mn.append(list_m)
    
fisher_matrix = np.array(list_mn)                
print(f'The Fisher Information Matrix, Fij = < del(-lnL)/deltheta_i deltheta_j> (theta = [H0, OmegaLambda, Omegam]), is: \n', fisher_matrix)                


The Fisher Information Matrix, Fij = < del(-lnL)/deltheta_i deltheta_j> (theta = [H0, OmegaLambda, Omegam]), is: 
 [[3.95489770e+00 1.28976048e+02 2.53194169e+02]
 [1.28976048e+02 4.24149523e+03 8.67376776e+03]
 [2.53194169e+02 8.67376776e+03 2.12137038e+04]]


### Confidence Region

### B) Likelihood (-2ln L)  and best fit (with all parameters free)

In [7]:
def Dl_all(z, theta):
#Theta
    H0, Omega_m0, Omega_k0, Omega_l0, Omega_r0  = theta[0], theta[1], theta[2], theta[3], theta[4]

#Hubble function
    E2z = Omega_l0 + Omega_k0 * ((1 + z) ** 2) + Omega_m0 * ((1 + z) ** 3) + Omega_r0 * ((1 + z) ** 4) 
    Ez = E2z ** 0.5
    
#Comoving distance
    def integ(b):

        int_arg = lambda z: Ez
        result = quad(int_arg, 0, b)
        return result[0]

    dc_list = []
    for i in range(lenz):
        d = integ (z[i])     #(* c/H0)   
        dc_list.append(d)

    dc = np.array(dc_list)      
    Dc =  dc     #( * H0/c)

# Temporal comoving distance
    if Omega_k0 == 0:
        Dt = Dc
    else:
        Dt = (sinh((Omega_k0 ** 0.5)) * Dc) / (Omega_k0 ** 0.5)
        
#Luminosity distance
    D_l = (1 + z) * Dt

    return D_l



def mu_all(z, theta): #This function calculates the modular distance

    H0 =theta[0]
    
    DL = Dl_all(z, theta)
    mu = ( 5 * log10(DL) ) + 25 + ( 5 * log10(c / H0))
    
    return mu 



def f_2lnL_all(theta): #This function calculates -2lnL (L is the Likelihood).
    arg = []
    for i in range(lenz):
        argi = ( ( mu_all(zobs,theta)[i] - muobs[i] ) ** 2 ) / ( sigmaobs[i] ** 2 )
        arg.append(argi)
        
    return sum(arg)  


In [8]:
#Best fit (all parameters free)

def fit_all():
    x0 = [70.0, 0.02, 0.20, 0.80, 1e-05]
    x0_np = np.array(x0)
    bnds = ((0, None),(0, None), (0, 1.0), (0, 1.0),(0, None))
    
    f = minimize(f_2lnL, x0_np, bounds = bnds, method='Nelder-Mead', tol=1e-2)
    
    return f

fit_all = fit_all()

print(fit_all) 

 final_simplex: (array([[1.64025754e+02, 4.78797349e-02, 1.34580961e-01, 5.89353336e-01,
        6.27107121e-06],
       [1.64033847e+02, 4.78842802e-02, 1.34551370e-01, 5.89359109e-01,
        6.27097875e-06],
       [1.64022395e+02, 4.78775797e-02, 1.34592686e-01, 5.89352898e-01,
        6.27113491e-06],
       [1.64029023e+02, 4.78823906e-02, 1.34562883e-01, 5.89367086e-01,
        6.27101490e-06],
       [1.64019218e+02, 4.78764385e-02, 1.34598503e-01, 5.89357194e-01,
        6.27123281e-06],
       [1.64029763e+02, 4.78824729e-02, 1.34559222e-01, 5.89362748e-01,
        6.27122007e-06]]), array([110.99870541, 110.99870598, 110.9987067 , 110.99870692,
       110.99870852, 110.99870902]))
           fun: 110.9987054121402
       message: 'Optimization terminated successfully.'
          nfev: 263
           nit: 149
        status: 0
       success: True
             x: array([1.64025754e+02, 4.78797349e-02, 1.34580961e-01, 5.89353336e-01,
       6.27107121e-06])


In [9]:
#Fisher Matrix (all parameters free)

#Parameters and data
theta = fit_all.x
H0 = theta[0]
z = zobs

# factor: del(dc)/delOmega_lambda/m
def dOm(z, Omega):
    dc_list = []
    for i in range(lenz):
        
        H0, Omega_m0, Omega_k0, Omega_l0, Omega_r0  = theta[0], theta[1], theta[2], theta[3], theta[4]
        
        if Omega == 1:        #Omega_Lambda case
            int_arg = lambda z: -(0.5) * ((Omega_l0 + (Omega_m0 * ((1 + z) ** 3)))) ** (3/2)
            d = quad(int_arg, 0, z[i])
        
        elif Omega == 2:   #Omega_m case
            int_arg = lambda z: -((1 + z) ** 3) * (0.5) * ((Omega_l0 + (Omega_m0 * ((1 + z) ** 3)))) ** (3/2)
            d = quad(int_arg, 0, z[i])
        
        elif Omega == 3:   #Omega_k case
            int_arg = lambda z: -((1 + z) ** 2) * (0.5) * ((Omega_l0 + (Omega_m0 * ((1 + z) ** 3)))) ** (3/2)
            d = quad(int_arg, 0, z[i])
        
        else:                 #Omega_l case
            int_arg = lambda z: -((1 + z) ** 4) * (0.5) * ((Omega_l0 + (Omega_m0 * ((1 + z) ** 3)))) ** (3/2)
            d = quad(int_arg, 0, z[i])
        
        dc_list.append(d[0])

    return np.array(dc_list)


#delmu/delH0
delmuH0 =  []

#delmu/delOmegalambda
delmuOL = []

#delmu/delOmegam
delmuOM = []

#delmu/delOmegam
delmuOK = []

#delmu/delOmegam
delmuOR = []

#This calculates the derivatives
for k in range(lenz):
    delmuH0i =(- 5) / (H0 * ln(10))
    delmuOLi = (5 / (Dl(z, theta)[k] * ln(10)) * dOm(zobs, 1)[k])
    delmuOMi = (5 / (Dl(z, theta)[k] * ln(10)) * dOm(zobs, 2)[k])
    delmuOKi = (5 / (Dl(z, theta)[k] * ln(10)) * dOm(zobs, 3)[k])
    delmuORi = (5 / (Dl(z, theta)[k] * ln(10)) * dOm(zobs, 4)[k])
       
    delmuH0.append(delmuH0i)
    delmuOL.append(delmuOLi)
    delmuOM.append(delmuOMi)
    delmuOK.append(delmuOKi)
    delmuOR.append(delmuORi)

#list of derivatives from mu
der = [delmuH0, delmuOL, delmuOM, delmuOK, delmuOR] 

#Fisher Matrix
list_mn = []
for n in der:
    list_m = []
    for m in der:
        list_d = []
        for i in range(lenz):
            di = (1 / (sigma[i]) ** 2) * ( n[i] * m[i])
            list_d.append(di)
        list_m.append(sum(list_d))
    list_mn.append(list_m)
    
fisher_matrix = np.array(list_mn)                
print(f'The Fisher Information Matrix, Fij = < del(-lnL)/deltheta_i deltheta_j> \n (theta = [H0, Omega_k0, Omega_Lambda0, Omega_m0,Omega_r0]), is: \n \n', fisher_matrix)                


The Fisher Information Matrix, Fij = < del(-lnL)/deltheta_i deltheta_j> 
 (theta = [H0, Omega_k0, Omega_Lambda0, Omega_m0,Omega_r0]), is: 
 
 [[7.91187954e-01 1.25826741e+01 2.20589071e+01 1.77689968e+01
  2.82774995e+01]
 [1.25826741e+01 2.01281260e+02 3.42916951e+02 2.78934896e+02
  4.35466110e+02]
 [2.20589071e+01 3.42916951e+02 6.90109020e+02 5.29859272e+02
  9.28135915e+02]
 [1.77689968e+01 2.78934896e+02 5.29859272e+02 4.14874020e+02
  6.99122375e+02]
 [2.82774995e+01 4.35466110e+02 9.28135915e+02 6.99122375e+02
  1.27136498e+03]]


### D) Likelihood ratio test

In [10]:
theta1 = [73.36421072,  0.23935098,  0.67269584] #Omegak0, Omegam0 = 0
theta2 = [1.64025754e+02, 4.78797349e-02, 1.34580961e-01, 5.89353336e-01,
       6.27107121e-06]

L1 = f_2lnL(theta1)
L2 = f_2lnL(theta2)

l = L1/L2

print(l)


0.9999999941681778
